# Sighash flags


## Background
- Raghav Sood has an excellent [blog](https://raghavsood.com/blog/2018/06/10/bitcoin-signature-types-sighash) covering the different sighash flags, their explanations and some examples of how they might be used.
- Note that taproot defines a [new sighash](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#common-signature-message) `SIGHASH_DEFAULT` (`0x00`). This sighash flag signs over the whole transaction as with `SIGHASH_ALL`, the only difference is that the sighash is not appended to the signature in the transaction, saving a byte.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'


## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *
from functions.bip_0340_reference import *

import json

## Scenario

### Problem
Jose is running a restaurant in El Salvador where most customers pay in bitcoin over the lightning network. He uses a hot wallet for his day to day business expenses, and a cold wallet for storing longer term savings. The signing device for his cold wallet is stored in a safe in his home town that he visits twice a year. 

It's Christmas and Jose is visiting home. He reflects on the past year of business and decides to relocate his restaurant to a larger location that an serve more customers. He knows he'll need to move some funds out of his cold storage but he'll have to go back to his restaurant before he figures out who he'll pay for the contract. The problem he faces is that he'd like to use the funds from his cold wallet a few weeks in the future when he's back at his restaurant, but it's only convenient for him to spend from it while he's at home with the offline signer.

He could create a transaction to transfer funds to his hot wallet, but he feels uncomfortable with relying on the hot wallet for storing larger amounts of bitcoin. What other option does Jose have?

### Solution
To solve this problem, uses a two step solution using a transaction that spends one input from his cold wallet (then amount he wants to transfer), and a second input from his hot wallet.

In the first step, while he is at home with his offline signer, he creates a partially signed transaction that spends from a large UTXOs from his cold wallet and a small UTXO from his hot wallet. He only creates a signature to spend the large input from the cold wallet, leaving the other input to sign later. Importantly, he uses the `SIGHASH_NONE` sighash flag as it does not commit to any outputs, meaning he can set it to contractor's address later. 

In the second step, after he has figured out who he will pay for the contract, he creates the rest of the transaction by filling in the outputs, and creating a signature on the second input. For the second input he creates a signature using the `SIGHASH_ALL` flag, committing to all the outputs

## Setup

To begin this scenario, we'll create two UTXOs for Jose, one representing a UTXO from his cold wallet, and another from his hot (lightning) wallet.

In [2]:
# Create addresses for Jose's cold and hot wallet UTXOs that we'll fund as part of the setup.
cold_privkey = bytes.fromhex("11f9ed113402e4969b711c5dc4cb750be6c022e9958a9a01fbd10bd68176bf92")
cold_pubkey = tr.pubkey_gen(cold_privkey)
cold_spk = bytes.fromhex("5120") + cold_pubkey
cold_address = spk_to_bech32(cold_spk, 'regtest')

hot_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
hot_pubkey = tr.pubkey_gen(hot_privkey)
hot_spk = bytes.fromhex("5120") + hot_pubkey
hot_address = spk_to_bech32(hot_spk, 'regtest')

Now we are ready to set up the bitcoin `TestShell` and fund the addresses. 

In [3]:
# Setup bitcoind and fund the addresses
node = setup_testshell()
cold_txid, cold_index = fund_address(node, cold_address, 2.001)
hot_txid, hot_index = fund_address(node, hot_address, 0.101)
print(f"Cold wallet UTXO: {cold_txid}, {cold_index}")
print(f"Hot wallet UTXO: {hot_txid}, {hot_index}")

2022-12-08T11:02:52.304000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_iwmc14gg
Cold wallet UTXO: 6697e6a4cb3d06f4c8342583d8158a51513aedfa404514f4af740a222371431d, 0
Hot wallet UTXO: 57bc2605d0218b4709f3eed074e998b51dd6998723a1024002f51df40c3f5044, 1


## Part 1: Sign the first input with SIGHASH_NONE

In the first step, Jose creates a partial transaction defining all the fields apart from the outputs. The main purpose of this first step is to create a signature which can then be used to complete the final transaction once the outputs are known.

In [4]:
# VERSION
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
input_count = bytes.fromhex("02")

# Convert txid and index to bytes (little endian)
c_txid = (bytes.fromhex(cold_txid))[::-1]
c_index = cold_index.to_bytes(4, byteorder="little", signed=False)
h_txid = (bytes.fromhex(hot_txid))[::-1]
h_index = hot_index.to_bytes(4, byteorder="little", signed=False)

# For segwit transactions the scriptSig is left empty
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    c_txid
    + c_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
    + h_txid
    + h_index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

Now that we've defined all the fields apart from the output, we are ready to create a signature over it for the cold wallet's UTXO using the `SIGHASH_NONE` flag.

In [5]:
sighash_epoch = bytes.fromhex("00")

# The cold wallet UTXO is the first input, so we set this to 0
index_of_this_input = bytes.fromhex("0000 0000")

# Control
cold_hash_type = bytes.fromhex("02") # SIGHASH_NONE

# Transaction data
sha_prevouts = sha256(
    c_txid 
    + c_index
    + h_txid
    + h_index
)

cold_input_amount_sat = int(2.001 * 100_000_000)
cold_input_amount = cold_input_amount_sat.to_bytes(8, byteorder="little", signed=False)
hot_input_amount_sat = int(0.101 * 100_000_000)
hot_input_amount = hot_input_amount_sat.to_bytes(8, byteorder="little", signed=False)
# Note that sha_amounts is of the serialization of all input amounts, not the summed total
sha_amounts = sha256(cold_input_amount + hot_input_amount)

sha_scriptpubkeys = sha256(
    varint_len(cold_spk)
    + cold_spk
    + varint_len(hot_spk)
    + hot_spk
)

sha_sequences = sha256(sequence + sequence)

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + cold_hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
#     + sha_outputs  # SIGHASH_NONE means we are not signing over any outputs!
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
cold_sighash = sha256(tag_hash + tag_hash + sig_msg)
print(cold_sighash.hex())

18b9b42c3ce105eccf440ae37be7794003358a659b0c9b57b4d1cb6581ac2a61


In [6]:
# Sign the partial transaction
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
cold_signature = tr.schnorr_sign(cold_sighash, cold_privkey, aux_rand)

# Append the sighash flag 
cold_signature += cold_hash_type

print("Signature for the cold wallet UTXO: ", cold_signature.hex())

Signature for the cold wallet UTXO:  493b33b734db4e9a779b92c1f224590db29c2c960ba08ff67fcadfbe9073b1f39c995c7cf604e3ac9a6cd12bf26297a9573d2083c54db5a36a85f34221d9ea6a02


Now that Jose has the signature for the cold wallet UTXO, he can take it with him back when he goes back to his restaurant. As long as he's confident that his hot wallet private keys have not been compromised, he will be able to complete the second half of the transaction securely.

## Part 2: Signing the second input with SIGHASH_ALL

Jose has had time to find a suitable contractor for his restaurant's relocation, has negotiated a price of 1.6 bitcoins and is ready to pay. He can now define the outputs for his partial bitcoin transaction and sign the second input using the lighting hot wallet.

In [8]:
# Decode the receiver's address
contractors_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'
hrp = 'bcrt'
witver, witprog = b32.decode(hrp, contractors_address)
pubkey_hash = bytearray(witprog)
receiver_spk = bytes.fromhex("0014") + pubkey_hash

# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Set our output amounts (in satoshis) and scriptPubkeys
output1_value_sat = int(float("1.6") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

In [9]:
# OUTPUTS
# 0x02 for two outputs
output_count = bytes.fromhex("02")
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000021d437123220a74aff4144540faed3a51518a15d8832534c8f4063dcba4e697660000000000ffffffff44503f0cf41df5024002a1238799d61db598e974d0eef309478b21d00526bc570100000000ffffffff020068890900000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


Above is the parsable unsigned transaction that contains all the inputs and outputs. We already have the signature for the first input `cold_signature`. Now all we need is to create the signature for the second input and then add them to the witness.

In [10]:
# Create Signature for Hot Wallet UTXO
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0100 0000")

# Control
hot_hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT

# SIGHASH_DEFAULT means we are signing over all outputs
sha_outputs = sha256(outputs)

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

# The other fields (sha_prevouts, sha_amounts,...etc) are the same as before
sig_msg = (
    sighash_epoch
    + hot_hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
hot_sighash = sha256(tag_hash + tag_hash + sig_msg)

aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
hot_signature = tr.schnorr_sign(hot_sighash, hot_privkey, aux_rand)

# Note that we don't append the SIGHASH_DEFAULT flag to the signature
print("Signature for the hot wallet UTXO: ", hot_signature.hex())

Signature for the hot wallet UTXO:  cfb72ddd898c4ead3f932655c5e8da293ff9405e78653cb9bbd33482cf9ce01689e2f31522549a5cc7e028553adc00572bc8be22f41533a8484b7127ae50a3cb


Nearly there! All we need to do is define the witness field with both signatures.

In [11]:
witness = (
    bytes.fromhex("01") # one stack item in the witness for input 1
    + varint_len(cold_signature)
    + cold_signature
    + bytes.fromhex("01") # one stack item in the witness for input 2
    + varint_len(hot_signature)
    + hot_signature
)

# the final signed transaction with both signature
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  020000000001021d437123220a74aff4144540faed3a51518a15d8832534c8f4063dcba4e697660000000000ffffffff44503f0cf41df5024002a1238799d61db598e974d0eef309478b21d00526bc570100000000ffffffff020068890900000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d0141493b33b734db4e9a779b92c1f224590db29c2c960ba08ff67fcadfbe9073b1f39c995c7cf604e3ac9a6cd12bf26297a9573d2083c54db5a36a85f34221d9ea6a020140cfb72ddd898c4ead3f932655c5e8da293ff9405e78653cb9bbd33482cf9ce01689e2f31522549a5cc7e028553adc00572bc8be22f41533a8484b7127ae50a3cb00000000


Now we can verify the validity of the transaction by running `testmempoolaccept` with bitcoind.

In [12]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': '51eb1f909586cad13a8778301cfe444145cbff4407789bdc17b6035e0f0c24a6', 'wtxid': '5e9b4f4e82eff99b4276661c1d00986b69b26788a41a62bf0f06b389df7a72d5', 'allowed': True, 'vsize': 188, 'fees': {'base': Decimal('0.00200000')}}]


## Quiz

1. In this example, if Jose accidentally leaked his partially signed transaction after step 1, what would prevent someone else from using that signature signed with the `SIGHASH_NONE` flag (`cold_signature`) from adding it to their own transaction?  
2. If Jose suspected that an attacker had compromised his lightning wallet private keys, could he safely attempt to broadcast the same transaction (`signed_tx`) anyway?
3. If Jose knew the amount he was going to pay the contractor, but not their address, what sighash flag could he use to constrain the final transaction further. What advantage if any might this have?

## Answers

1. The signature still signs over all the inputs. It would only ever be valid in a transaction that spent the same set of input UTXOs, so that spender would need knowledge of the first UTXO's private key.
2. No. If someone knew the lightning wallet's private key, they would be able to view the transaction once it was broadcasted, then create another signed transaction sending the total amount to themselves. Note that not opting in to rbf may help prevent an attacker from spending from the same input once the initial transaction has been broadcast, but it is not a guarantee.
3. He could use `SIGHASH_SINGLE` and sign over just the change output. It would help avoid losing all his funds in a worst case scenario where an attacker knew his lighting wallet private keys (as in Q2).

## Exercise

As in Q3 from the quiz, try editing the example above to set the first input to use `SIGHASH_SINGLE` instead of `SIGHASH_NONE`.